# Analiza Danych - Projekt
Marcin Jeznach 241204 \
Semestr letni 2023/24

## Zbiór danych

[Statlog (German Credit Data)](https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data "Zbiór danych - UCI")

Zbiór danych zawiera dane 1000 klientów, będących kredytobiorcami banku i klasyfikuje ich na dwie klasy (dobrych kredytobiorców i złych kredytobiorców, lub inaczej niskie i wysokie ryzyko). Celem projektu jest stworzenie modelu predykcji ryzyka kredytowego.

### Opis kolumn

Zarówno w opisie zbiory danych, jak i samym zbiorze danych, kolumny nie mają opisowych nazw - opis wymienia jedynie nazwy `Attribute 1` - `Attribute 20`. Aby zwiększyć czytelność zarówno kodu, jak i tabel/wykresów, do danych przypisane zostały własne nazwy.

Wartości kolumn kategorycznych zostały zakodowane w danych źródłowych jako `A{n}{m}`, gdzie `{n}` to numer kolumny, a `{m}` to numer kategorii. Aby ponownie zwiększyć czytelność, jako słowniki dla tych kategorii, użyte zostaną opisy wartości, przetłumaczone na język polski.

Warto również zwrócić uwagę, że zbiór danych powstał w 1994 roku, przez co wartości walutowe podawane są w Markach Niemieckich (DM).

Jeżeli nie podano inaczej, to wartości czasowe są najprawdopodobniej wyrażone w latach.

| Nazwa kolumny (oryginalna) | Nazwa kolumny (własna)    | Typ kolumny  | Opis kolumny                                                |
| -------------------------- | ------------------------- | ------------ | ----------------------------------------------------------- |
| Attribute 1                | Saldo ROR                 | kategoryczna | Bieżące saldo używanego rachunku rozliczeniowego            |
| Attribute 2                | Okres kredytowania        | numeryczna   | Czas trwania kredytu, w miesiącach                          |
| Attribute 3                | Historia kredytowa        | kategoryczna | Historia dotychczasowych kredytów i terminowosci            |
| Attribute 4                | Cel konsumpcyjny          | kategoryczna | Na co planowane jest przeznaczenie pożyczonych środków      |
| Attribute 5                | Kwota kredytu             | numeryczna   |                                                             |
| Attribute 6                | Saldo oszczędności        | kategoryczna | Suma salda rachunków oszczędnościowych i wartości obligacji |
| Attribute 7                | Bieżący staż pracy        | kategoryczna | Czas pracy w bieżącym miejscu zatrudnienia                  |
| Attribute 8                | Wskaźnik RdD              | numeryczna   | Stosunek wysokości raty miesięcznej do dochodów, w %        |
| Attribute 9                | Stan cywilny              | kategoryczna |                                                             |
| Attribute 10               | Dodatkowy dłużnik         | kategoryczna | Czy do kredytu dopisane są inne osoby / w jakim charakterze |
| Attribute 11               | Bieżący czas zamieszkania | numeryczna   | Czas zamieszkania pod obecnym adresem zamieszkania          |
| Attribute 12               | Mienie                    | kategoryczna | Inne mienie nie wymienione w saldzie oszczędności           |
| Attribute 13               | Wiek                      | numeryczna   |                                                             |
| Attribute 14               | Pozostałe zobowiązania    | kategoryczna | Czy klient posiada inne zobowiązania ratalne                |
| Attribute 15               | Typ zamieszkania          | kategoryczna | Charakter w jakim klient zamieszkuje pod obecnym adresem    |
| Attribute 16               | Liczba zobowiązań         | numeryczna   | Liczba pozostałych zobowiązań                               |
| Attribute 17               | Rodzaj zawodu             | kategoryczna |                                                             |
| Attribute 18               | Osoby na utrzymaniu       | numeryczna   | Liczba osób będących na utrzymaniu klienta                  |
| Attribute 19               | Numer telefonu            | kategoryczna | Czy klient posiada / podał(a) numer telefonu                |
| Attribute 20               | Pracownik zagraniczny     | kategoryczna | Czy klient jest pracownikiem zagranicznym                   |

Dodatkowo występuje kolumna kategoryczna z oceną ryzyka kredytowego.


#### Załadowanie danych

In [2]:
import pandas as pd
import seaborn as sns
import scipy.stats as sp_stats

data = pd.read_csv(
	"./german.data",
	sep=" ", 
	names= [
		"Saldo ROR",
		"Okres kredytowania",
		"Historia kredytowa",
		"Cel konsumpcyjny",
		"Kwota kredytu",
		"Saldo oszczędności",
		"Bieżący staż pracy",
		"Wskaźnik RdD",
		"Stan cywilny",
		"Dodatkowy dłużnik",
		"Bieżący czas zamieszkania",
		"Mienie",
		"Wiek",
		"Pozostałe zobowiązania",
		"Typ zamieszkania",
		"Liczba zobowiązań",
		"Rodzaj zawodu",
		"Osoby na utrzymaniu",
		"Numer telefonu",
		"Pracownik zagraniczny",
		"Ocena ryzyka"
	],
	index_col= False
)

### Słowniki kolumn kategorycznych

Poniżej znajdują się mapowania znaczenia kolumn kategorycznych. Ewentualne objaśnienia znajdują się w komentarzu na końcu linii z daną wartością.

Jeżeli nie podano inaczej, to górna wartość zakresów jest wartością wyłączną (wartość `v` w zakresie `x` do `y` oznacza `x` <= `v` < `y`).

In [3]:

data["Saldo ROR"] = data["Saldo ROR"].map({
	"A11": "Poniżej 0 DM",
	"A12": "0 DM do 200 DM",
	"A13": "Ponad 200 DM",
	"A14": "Brak ROR",
})

data["Historia kredytowa"] = data["Historia kredytowa"].map({
	"A30": "Nigdy nie brano lub Wszystkie spłacono w terminach",
	"A31": "Wszystkie kredyty wewnętrzne spłacono w terminie", # Przez kredyty wewnętrzne danego banku, rozumie się takie których dany bank jest kredytodawcą
	"A32": "Istniejące kredyty dotychczasowo spłacono w terminie",
	"A33": "W przeszłości występowały opóźnienia w spłacie",
	"A34": "Krytyczny rachunek lub istniejące kredyty zewnętrzne",
})

data["Cel konsumpcyjny"] = data["Cel konsumpcyjny"].map({
	"A40": "Samochód nowy",
	"A41": "Samochód używany",
	"A42": "Meble i wyposażenie",
	"A43": "RTV",
	"A44": "AGD",
	"A45": "Naprawy",
	"A46": "Edukacja",
	"A47": "Wakacje",
	"A48": "Szkolenia", # Wartość oryginalna "retraining", może również oznaczać przekwalifikowanie
	"A49": "Wydatki biznesowe",
	"A410": "Inne",
})

data["Saldo oszczędności"] = data["Saldo oszczędności"].map({
	"A61": "Poniżej 100 DM",
	"A62": "100 DM do 500 DM",
	"A63": "500 DM do 1000 DM",
	"A64": "Powyżej 1000 DM",
	"A65": "Brak lub nieznana wartość",
})

data["Bieżący staż pracy"] = data["Bieżący staż pracy"].map({
	"A71": "Bezrobotny/a",
	"A72": "Do 1 roku",
	"A73": "1 do 4 lat",
	"A74": "4 do 7 lat",
	"A75": "Powyżej 7 lat",
})

data["Stan cywilny"] = data["Stan cywilny"].map({
	"A91": "Rozwodnik lub w separacji",
	"A92": "Rozwódka, w separacji lub zamężna",
	"A94": "Kawaler",
	"A93": "Żonaty lub wdowiec",
	"A95": "Panna",
})

data["Dodatkowy dłużnik"] = data["Dodatkowy dłużnik"].map({
	"A101": "Brak",
	"A102": "Współkredytobiorca",
	"A103": "Poręczyciel",
})

data["Mienie"] = data["Mienie"].map({
	"A121": "Nieruchomości",
	"A122": "Oszczędności w Towarzystwie Budowlanym lub Ubezpieczenie na życie",
	"A123": "Samochód lub inne (nie dotyczy oszczędności)",
	"A124": "Nieznane lub brak majątku nieruchomego",
})

data["Pozostałe zobowiązania"] = data["Pozostałe zobowiązania"].map({
	"A141": "Kredyty bankowe",
	"A142": "Zakupy na raty",
	"A143": "Brak",
})

data["Typ zamieszkania"] = data["Typ zamieszkania"].map({
	"A151": "Wynajem",
	"A152": "Nieruchomość na własność",
	"A153": "Mieszkanie socjalne", # Wartość oryginalna to "for free", ciężko znaleźć inne sensowne tłumaczenie
})

data["Rodzaj zawodu"] = data["Rodzaj zawodu"].map({
	"A171": "Bezrobotny lub niewykwalifikowany/a - brak obywatelstwa",
	"A172": "Niewykwalifikowany/a - obywatel(ka)",
	"A173": "Wykwalifikowany/a lub Urzędnik/czka",
	"A174": "Kadra menedżerska, Samozatrudnienie, Wysokie kwalifikacje lub Funkcjonariusz(ka)",
})

data["Numer telefonu"] = data["Numer telefonu"].map({
	"A191": "Brak numeru telefonu",
	"A192": "Numer telefonu zarejestrowany na danego klienta",
})

data["Pracownik zagraniczny"] = data["Pracownik zagraniczny"].map({
	"A201": "Tak",
	"A202": "Nie",
})

data["Ocena ryzyka"] = data["Ocena ryzyka"].map({
	1: "Niskie ryzyko / dobry klient",
	2: "Wysokie ryzyko / zły klient",
})

data.head()

,Saldo ROR,Okres kredytowania,Historia kredytowa,Cel konsumpcyjny,Kwota kredytu,Saldo oszczędności,Bieżący staż pracy,Wskaźnik RdD,Stan cywilny,Dodatkowy dłużnik,...,Mienie,Wiek,Pozostałe zobowiązania,Typ zamieszkania,Liczba zobowiązań,Rodzaj zawodu,Osoby na utrzymaniu,Numer telefonu,Pracownik zagraniczny,Ocena ryzyka
0,Poniżej 0 DM,6,Krytyczny rachunek lub istniejące kredyty zewn...,RTV,1169,Brak lub nieznana wartość,Powyżej 7 lat,4,Żonaty lub wdowiec,Brak,...,Nieruchomości,67,Brak,Nieruchomość na własność,2,Wykwalifikowany/a lub Urzędnik/czka,1,Numer telefonu zarejestrowany na danego klienta,Tak,Niskie ryzyko / dobry klient
1,0 DM do 200 DM,48,Istniejące kredyty dotychczasowo spłacono w te...,RTV,5951,Poniżej 100 DM,1 do 4 lat,2,"Rozwódka, w separacji lub zamężna",Brak,...,Nieruchomości,22,Brak,Nieruchomość na własność,1,Wykwalifikowany/a lub Urzędnik/czka,1,Brak numeru telefonu,Tak,Wysokie ryzyko / zły klient
2,Brak ROR,12,Krytyczny rachunek lub istniejące kredyty zewn...,Edukacja,2096,Poniżej 100 DM,4 do 7 lat,2,Żonaty lub wdowiec,Brak,...,Nieruchomości,49,Brak,Nieruchomość na własność,1,Niewykwalifikowany/a - obywatel(ka),2,Brak numeru telefonu,Tak,Niskie ryzyko / dobry klient
3,Poniżej 0 DM,42,Istniejące kredyty dotychczasowo spłacono w te...,Meble i wyposażenie,7882,Poniżej 100 DM,4 do 7 lat,2,Żonaty lub wdowiec,Poręczyciel,...,Oszczędności w Towarzystwie Budowlanym lub Ube...,45,Brak,Mieszkanie socjalne,1,Wykwalifikowany/a lub Urzędnik/czka,2,Brak numeru telefonu,Tak,Niskie ryzyko / dobry klient
4,Poniżej 0 DM,24,W przeszłości występowały opóźnienia w spłacie,Samochód nowy,4870,Poniżej 100 DM,1 do 4 lat,3,Żonaty lub wdowiec,Brak,...,Nieznane lub brak majątku nieruchomego,53,Brak,Mieszkanie socjalne,2,Wykwalifikowany/a lub Urzędnik/czka,2,Brak numeru telefonu,Tak,Wysokie ryzyko / zły klient


Dane nie posiadają wartości brakujących, ani identyfikatorów - dalsze przekształcenia nie są konieczne.

## Analiza danych

### Statystyki wartości numerycznych

In [11]:
data.select_dtypes("int").describe()

,Okres kredytowania,Kwota kredytu,Wskaźnik RdD,Bieżący czas zamieszkania,Wiek,Liczba zobowiązań,Osoby na utrzymaniu
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.155000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.362086
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000


#### Klienci z niskim ryzykiem

In [13]:
data[data["Ocena ryzyka"] == "Niskie ryzyko / dobry klient"].select_dtypes("int").describe()

,Okres kredytowania,Kwota kredytu,Wskaźnik RdD,Bieżący czas zamieszkania,Wiek,Liczba zobowiązań,Osoby na utrzymaniu
count,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,19.207143,2985.457143,2.920000,2.842857,36.224286,1.424286,1.155714
std,11.079564,2401.472278,1.128078,1.108373,11.381145,0.584721,0.362844
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000
25%,12.000000,1375.500000,2.000000,2.000000,27.000000,1.000000,1.000000
50%,18.000000,2244.000000,3.000000,3.000000,34.000000,1.000000,1.000000
75%,24.000000,3634.750000,4.000000,4.000000,42.250000,2.000000,1.000000
max,60.000000,15857.000000,4.000000,4.000000,75.000000,4.000000,2.000000


#### Klienci z wysokim ryzykiem

In [14]:
data[data["Ocena ryzyka"] == "Wysokie ryzyko / zły klient"].select_dtypes("int").describe()

,Okres kredytowania,Kwota kredytu,Wskaźnik RdD,Bieżący czas zamieszkania,Wiek,Liczba zobowiązań,Osoby na utrzymaniu
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,24.860000,3938.126667,3.096667,2.850000,33.963333,1.366667,1.153333
std,13.282639,3535.818955,1.088395,1.094605,11.222379,0.559702,0.360911
min,6.000000,433.000000,1.000000,1.000000,19.000000,1.000000,1.000000
25%,12.000000,1352.500000,2.000000,2.000000,25.000000,1.000000,1.000000
50%,24.000000,2574.500000,4.000000,3.000000,31.000000,1.000000,1.000000
75%,36.000000,5141.500000,4.000000,4.000000,40.000000,2.000000,1.000000
max,72.000000,18424.000000,4.000000,4.000000,74.000000,4.000000,2.000000


Rozkład ocen ryzyka klientów wynosi 700-300 (niskie-wysokie), co może potencjalnie wymagać balansowania.

Na pierwszy rzut oka, z wyjątkiem okresu kredytowania oraz kwoty kredytu, reszta kolumn numerycznych ma zbliżone wartości średnie.

### Tabele krzyżowe wartości kategorycznych

In [23]:
def get_crosstab(col):
	return pd.crosstab(
		data["Ocena ryzyka"],
		data[col],
		normalize="index"
	).style.background_gradient(
		axis= None,
		cmap= "pink",
	)
get_crosstab("Saldo ROR")

Saldo ROR,0 DM do 200 DM,Brak ROR,Ponad 200 DM,Poniżej 0 DM
Ocena ryzyka,,,,
Niskie ryzyko / dobry klient,0.234286,0.497143,0.070000,0.198571
Wysokie ryzyko / zły klient,0.350000,0.153333,0.046667,0.450000


In [25]:
get_crosstab("Historia kredytowa")

Historia kredytowa,Istniejące kredyty dotychczasowo spłacono w terminie,Krytyczny rachunek lub istniejące kredyty zewnętrzne,Nigdy nie brano lub Wszystkie spłacono w terminach,W przeszłości występowały opóźnienia w spłacie,Wszystkie kredyty wewnętrzne spłacono w terminie
Ocena ryzyka,,,,,
Niskie ryzyko / dobry klient,0.515714,0.347143,0.021429,0.085714,0.030000
Wysokie ryzyko / zły klient,0.563333,0.166667,0.083333,0.093333,0.093333


In [26]:
get_crosstab("Cel konsumpcyjny")

Cel konsumpcyjny,AGD,Edukacja,Inne,Meble i wyposażenie,Naprawy,RTV,Samochód nowy,Samochód używany,Szkolenia,Wydatki biznesowe
Ocena ryzyka,,,,,,,,,,
Niskie ryzyko / dobry klient,0.011429,0.040000,0.010000,0.175714,0.020000,0.311429,0.207143,0.122857,0.011429,0.090000
Wysokie ryzyko / zły klient,0.013333,0.073333,0.016667,0.193333,0.026667,0.206667,0.296667,0.056667,0.003333,0.113333


In [27]:
get_crosstab("Saldo oszczędności")

Saldo oszczędności,100 DM do 500 DM,500 DM do 1000 DM,Brak lub nieznana wartość,Poniżej 100 DM,Powyżej 1000 DM
Ocena ryzyka,,,,,
Niskie ryzyko / dobry klient,0.098571,0.074286,0.215714,0.551429,0.060000
Wysokie ryzyko / zły klient,0.113333,0.036667,0.106667,0.723333,0.020000


In [28]:
get_crosstab("Bieżący staż pracy")

Bieżący staż pracy,1 do 4 lat,4 do 7 lat,Bezrobotny/a,Do 1 roku,Powyżej 7 lat
Ocena ryzyka,,,,,
Niskie ryzyko / dobry klient,0.335714,0.192857,0.055714,0.145714,0.270000
Wysokie ryzyko / zły klient,0.346667,0.130000,0.076667,0.233333,0.213333


In [29]:
get_crosstab("Stan cywilny")

Stan cywilny,Kawaler,Rozwodnik lub w separacji,"Rozwódka, w separacji lub zamężna",Żonaty lub wdowiec
Ocena ryzyka,,,,
Niskie ryzyko / dobry klient,0.095714,0.042857,0.287143,0.574286
Wysokie ryzyko / zły klient,0.083333,0.066667,0.363333,0.486667


In [30]:
get_crosstab("Dodatkowy dłużnik")

Dodatkowy dłużnik,Brak,Poręczyciel,Współkredytobiorca
Ocena ryzyka,,,
Niskie ryzyko / dobry klient,0.907143,0.060000,0.032857
Wysokie ryzyko / zły klient,0.906667,0.033333,0.060000


In [31]:
get_crosstab("Mienie")

Mienie,Nieruchomości,Nieznane lub brak majątku nieruchomego,Oszczędności w Towarzystwie Budowlanym lub Ubezpieczenie na życie,Samochód lub inne (nie dotyczy oszczędności)
Ocena ryzyka,,,,
Niskie ryzyko / dobry klient,0.317143,0.124286,0.230000,0.328571
Wysokie ryzyko / zły klient,0.200000,0.223333,0.236667,0.340000


In [32]:
get_crosstab("Pozostałe zobowiązania")

Pozostałe zobowiązania,Brak,Kredyty bankowe,Zakupy na raty
Ocena ryzyka,,,
Niskie ryzyko / dobry klient,0.842857,0.117143,0.040000
Wysokie ryzyko / zły klient,0.746667,0.190000,0.063333


In [33]:
get_crosstab("Typ zamieszkania")

Typ zamieszkania,Mieszkanie socjalne,Nieruchomość na własność,Wynajem
Ocena ryzyka,,,
Niskie ryzyko / dobry klient,0.091429,0.752857,0.155714
Wysokie ryzyko / zły klient,0.146667,0.620000,0.233333


In [34]:
get_crosstab("Rodzaj zawodu")

Rodzaj zawodu,Bezrobotny lub niewykwalifikowany/a - brak obywatelstwa,"Kadra menedżerska, Samozatrudnienie, Wysokie kwalifikacje lub Funkcjonariusz(ka)",Niewykwalifikowany/a - obywatel(ka),Wykwalifikowany/a lub Urzędnik/czka
Ocena ryzyka,,,,
Niskie ryzyko / dobry klient,0.021429,0.138571,0.205714,0.634286
Wysokie ryzyko / zły klient,0.023333,0.170000,0.186667,0.620000


In [35]:
get_crosstab("Numer telefonu")

Numer telefonu,Brak numeru telefonu,Numer telefonu zarejestrowany na danego klienta
Ocena ryzyka,,
Niskie ryzyko / dobry klient,0.584286,0.415714
Wysokie ryzyko / zły klient,0.623333,0.376667


In [36]:
get_crosstab("Pracownik zagraniczny")

Pracownik zagraniczny,Nie,Tak
Ocena ryzyka,,
Niskie ryzyko / dobry klient,0.047143,0.952857
Wysokie ryzyko / zły klient,0.013333,0.986667


*Powyższe tabele krzyżowe zostały znormalizowane na poziomie wierszy, dając liczności wyrażone w procentach.*

Tabele krzyżowe dla stanu cywilnego, występowania dodatkowego dłużnika, oraz czy osoba jest pracownikiem zagranicznym, wskazują niezbyt różnice w licznościach danych grup - może to oznaczać małe powiązanie tych zmiennych.

W przypadku faktu posiadania numeru telefonu, różnice liczności są już małe, a w przypadku rodzaju wykonywanego zawodu - w większości grup znikome; można wstępnie założyć brak powiązania zmiennych do oceny ryzyka.

### Test Chi-kwadrat

In [37]:
chi_table = pd.DataFrame(columns= ["Kolumna", "Typ", "Chi^2", "P-val[%]"])

for col in data.columns:
	if col == "Ocena ryzyka": continue
	contingency = pd.crosstab(data[col], data["Ocena ryzyka"])
	chi_2, p, _, _ = sp_stats.chi2_contingency(contingency)
	typ = "num" if data[col].dtype == "int64" else "kat"
	chi_table.loc[
		len(chi_table),
		["Kolumna", "Typ", "Chi^2", "P-val[%]"]
	] = (col, typ, chi_2, p)

chi_table["P-val[%]"] *= 100

chi_table.style.background_gradient(
	axis= None,
	cmap= "summer",
	subset= "P-val[%]",
).format({
	"P-val[%]": "{:.2f}"
})

,Kolumna,Typ,Chi^2,P-val[%]
0,Saldo ROR,kat,123.720944,0.00
1,Okres kredytowania,num,78.886937,0.00
2,Historia kredytowa,kat,61.691397,0.00
3,Cel konsumpcyjny,kat,33.356447,0.01
4,Kwota kredytu,num,931.746032,38.66
5,Saldo oszczędności,kat,36.098928,0.00
6,Bieżący staż pracy,kat,18.368274,0.10
7,Wskaźnik RdD,num,5.476792,14.00
8,Stan cywilny,kat,9.605214,2.22
9,Dodatkowy dłużnik,kat,6.645367,3.61


Zgodnie z testem widać, że z wyjątkiem okresu kredytowania, zmienne numeryczne nie mają powiązania z ryzykiem kredytowym.

Poza tym, rodzaj zawodu oraz fakt posiadania numeru telefonu, również posiada *p-value* znacznie wyższe od 5% - one również nie będą miały powiązania.

### Wybór zmiennych do tworzenia modeli

In [5]:
model_cols_df = chi_table[chi_table["P-val[%]"] < 5.0]
model_cols = list(model_cols_df["Kolumna"])
model_cols.append("Ocena ryzyka")
model_cols

['Saldo ROR',
 'Okres kredytowania',
 'Historia kredytowa',
 'Cel konsumpcyjny',
 'Saldo oszczędności',
 'Bieżący staż pracy',
 'Stan cywilny',
 'Dodatkowy dłużnik',
 'Mienie',
 'Pozostałe zobowiązania',
 'Typ zamieszkania',
 'Pracownik zagraniczny',
 'Ocena ryzyka']

Pomimo że mamy dostarczone dwie wersje danych - z kodowanymi wartościami kategorycznymi (`A{n}{m}`), oraz drugą z numerycznymi wartościami kategorii, to do tworzenia modeli zastosujemy wbudowane w pandas kodowanie *Dummy-code*, które pozwoli na automatyczne kodowanie/dekodowanie.

In [6]:
model_data = data[model_cols]
model_data_dc = pd.get_dummies(model_data)
model_data_dc.head()

,Okres kredytowania,Saldo ROR_0 DM do 200 DM,Saldo ROR_Brak ROR,Saldo ROR_Ponad 200 DM,Saldo ROR_Poniżej 0 DM,Historia kredytowa_Istniejące kredyty dotychczasowo spłacono w terminie,Historia kredytowa_Krytyczny rachunek lub istniejące kredyty zewnętrzne,Historia kredytowa_Nigdy nie brano lub Wszystkie spłacono w terminach,Historia kredytowa_W przeszłości występowały opóźnienia w spłacie,Historia kredytowa_Wszystkie kredyty wewnętrzne spłacono w terminie,...,Pozostałe zobowiązania_Brak,Pozostałe zobowiązania_Kredyty bankowe,Pozostałe zobowiązania_Zakupy na raty,Typ zamieszkania_Mieszkanie socjalne,Typ zamieszkania_Nieruchomość na własność,Typ zamieszkania_Wynajem,Pracownik zagraniczny_Nie,Pracownik zagraniczny_Tak,Ocena ryzyka_Niskie ryzyko / dobry klient,Ocena ryzyka_Wysokie ryzyko / zły klient
0,6,False,False,False,True,False,True,False,False,False,...,True,False,False,False,True,False,False,True,True,False
1,48,True,False,False,False,True,False,False,False,False,...,True,False,False,False,True,False,False,True,False,True
2,12,False,True,False,False,False,True,False,False,False,...,True,False,False,False,True,False,False,True,True,False
3,42,False,False,False,True,True,False,False,False,False,...,True,False,False,True,False,False,False,True,True,False
4,24,False,False,False,True,False,False,False,True,False,...,True,False,False,True,False,False,False,True,False,True
